In [1]:
d = {'key':'value', 'key1':'value1'}

In [2]:
d['key']

'value'

In [3]:
d.key

AttributeError: 'dict' object has no attribute 'key'

In [4]:
from box import ConfigBox

In [5]:
d2 = ConfigBox({'key':'value', 'key1':'value1'}) #config box type dictionary

In [6]:
d2

ConfigBox({'key': 'value', 'key1': 'value1'})

In [8]:
d2.key1

'value1'

In [ ]:
# 1:06:25

In [1]:
def get_predict(x:int,y:int):
    return x*y

In [2]:
get_predict(x=2, y=3)

6

In [3]:
get_predict(x=3, y="3")

'333'

In [4]:
from ensure import ensure_annotations

In [5]:
@ensure_annotations
def get_predict(x:int,y:int):
    return x*y

In [6]:
get_predict(x=4, y="6")

EnsureError: Argument y of type <class 'str'> to <function get_predict at 0x0000020FCA627BA0> does not match annotation type <class 'int'>

In [1]:
%pwd

'd:\\a27_YEARS_OLD\\end_to_end_pro_chicken_disease_deepL\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\a27_YEARS_OLD\\end_to_end_pro_chicken_disease_deepL'

In [5]:
from dataclasses import dataclass
from pathlib import Path


#entities entities class name
@dataclass(frozen=True)
class DataIngestionConfig: # this is actually not a python class it is a dataclass which define return type of a python function
    root_dir: Path
    souce_URL: str
    local_data_file: Path
    unzip_dir: Path

    

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [15]:
class ConfiguraitonManager:
    def __init__(
            self, 
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig: #define return type of this function
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig( #assign return type of this function
            #reading all the config one by one  store the value in these variables
            root_dir=config.root_dir,
            souce_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        # thats how we write custom return type of the function  using entity which can be esily use that in my component
        return data_ingestion_config




In [10]:
#create data ingestion component

In [11]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [12]:
#create data ingestion component
class DataIngesion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.souce_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} has been downloaded ! with folowing info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(path=self.config.local_data_file)}")
        
    def extract_zip_file(self):
        """
        extract zip file to local data directory
        zip_file_path :str
        function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"zip file has been extracted successfully")



In [16]:
# create pipeline for dataingestion
try:
    config = ConfiguraitonManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngesion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.info(e)
    raise e

[2024-09-07 19:07:44,296: INFO: common: yaml file: config\config.yaml loadded successfully]
[2024-09-07 19:07:44,299: INFO: common: yaml file: params.yaml loadded successfully]
[2024-09-07 19:07:44,302: INFO: common: created directory at: artifacts]
[2024-09-07 19:07:44,303: INFO: common: created directory at: artifacts/data_ingestion]
[2024-09-07 19:07:47,231: INFO: 2424705162: artifacts/data_ingestion/data.zip has been downloaded ! with folowing info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7E6C:6EDEE:4CE2F:5FD66:66DC5665
Accept-Ranges: bytes
Date: Sat, 07 Sep 2024 13:34:32 GMT
Via: 1.1 varnish
X-Served-By: cache-ccu830032-CC